Import modules

In [1]:
from lib.datasets import get_stock_price,train_test_split
from lib.aug import apply_augmentations,parse_augmentations,sig_normal
import torch
from torch import nn
import torch.nn.functional as F
from lib.utils import set_seed
import signatory
from models.vae import VAE, VAE_train
from models.betavae import BetaVAE, BetaVAE_train
from models.infovae import InfoVAE, InfoVAE_train
from models.cvae import CVAE, CAVE_train
from lib.metrics import mmd_loss

In [2]:
set_seed(0)

The considered truncated signature degree.

In [3]:
sig_degree = [3,4,5]

In [4]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [{"name": "LeadLag"}],
    "device" : "cuda",
    "depth" : sig_degree[0],
}

Get path augmentations.

In [5]:
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))

Data preprocessing.
- Split training data and testing data.
- Apply path augmentations to original path.
- Calculate signature.
- Normalize signatures.
- Pass to CUDA.

In [6]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
print("x_real_train shape {}, x_real_test shape {}".format(x_real_train.shape,x_real_test.shape))

if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_train, x_aug_test = apply_augmentations(x_real_train,sig_config["augmentations"]), apply_augmentations(x_real_test,sig_config["augmentations"])
print("After augmentation shape:",x_aug_train.shape)

# To signature
x_sig_train, x_sig_test = signatory.signature(x_aug_train,sig_config["depth"]),signatory.signature(x_aug_test,sig_config["depth"])
print("x_sig_train shape {}, x_sig_test shape {}".format(x_sig_train.shape,x_sig_test.shape))
input_dim = x_sig_train.shape[1]
print("input_dim: {}".format(input_dim))

# Normalize
x_sig_train, x_sig_test = sig_normal(x_sig_train,True), sig_normal(x_sig_test,True)

# To device
x_sig_train, x_sig_test = x_sig_train.to(sig_config["device"]), x_sig_test.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
x_real_train shape torch.Size([985, 20, 1]), x_real_test shape torch.Size([247, 20, 1])
torch.Size([985, 39, 2])
torch.Size([247, 39, 2])
After augmentation shape: torch.Size([985, 39, 2])
x_sig_train shape torch.Size([985, 14]), x_sig_test shape torch.Size([247, 14])
input_dim: 14


In [7]:
# For degree = 3
hidden_dim_degree3 = [input_dim,7,3] # input_dim = 14
hidden_dim_degree4 = [input_dim,15,7] # input_dim = 30
hidden_dim_degree5 = [input_dim,32,16] # input_dim = 62

Initialize models

In [8]:
# VAE
model_vae = VAE(x_aug_sig=x_sig_train,epoch=6000,batch_size=128,device="cuda",hidden_dims=hidden_dim_degree3)
optimizer = torch.optim.Adam(model_vae.parameters(), lr=1e-4)
VAE_train(model_vae,optimizer=optimizer)

Input tensor shape: torch.Size([985, 14])
Hidden dims: [14, 7, 3]
Epoch 0 loss 1.204512
Epoch 100 loss 0.508713
Epoch 200 loss 0.569742
Epoch 300 loss 1.317153
Epoch 400 loss 0.485645
Epoch 500 loss 0.650923
Epoch 600 loss 0.456017
Epoch 700 loss 0.400280
Epoch 800 loss 0.539389
Epoch 900 loss 0.977718
Epoch 1000 loss 1.048061
min_loss: 0.305935


In [9]:
# Beta-VAE
model_betavae = BetaVAE(x_sig_train,6000,128,beta=3,device='cuda',hidden_dims=hidden_dim_degree3)
optimizer = torch.optim.Adam(model_betavae.parameters(), lr=1e-4)
BetaVAE_train(model=model_betavae,optimizer=optimizer)

Input tensor shape: torch.Size([985, 14])
Hidden dims: [14, 7, 3]
Beta: 3
Epoch 0 loss 31.404409
Epoch 100 loss 20.957376
Epoch 200 loss 32.084732
Epoch 300 loss 8.619654
Epoch 400 loss 9.748044
min_loss: 5.136885
